In [44]:
# import os
# import wandb
# os.environ['WANDB_USERNAME']="qiker-chen"
# wandb.login(key="69fdb45d404436728fa8b67771c5e3b4b94e6002")

In [2]:
model_path="/home/ckqsudo/code2024/0model/Mistral-7B-Instruct-v0.2"
import os
os.environ["WANDB_API_KEY"]="284e19ed3e365d69a51b9c3b4d61aa9f72dfa2ed"
os.environ["ALL_PROXY"]="127.0.0.1:11137"
import wandb


In [46]:
# http://localhost:8888/tree?token=6ef876332060f7c3d066571cd2918641b875b9cd557cac0e

In [3]:
# import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3"
# http://localhost:8888/tree?token=6ef876332060f7c3d066571cd2918641b875b9cd557cac0e

In [4]:

dataset_arg="_continue_training"
# /dataset_arg="+reasoning"

In [5]:
from random import shuffle
from datasets import load_from_disk
from datasets import load_dataset
# Load jsonl data from disk
from datasets import DatasetDict
from datasets import Dataset
from datasets import concatenate_datasets
dataset_values=["_observation_reasoning","_reasoning","_pure_dialog","_pure_reasoning_observation","_continue_training"]

if dataset_arg not in dataset_values:
    raise ValueError()
if dataset_arg=="_observation_reasoning":
    en_dataset=load_from_disk("/home/ckqsudo/code2024/Mistral-7b-finetune/hf_en_observation_reasoning_dataset_3")
    dataset=en_dataset.train_test_split(0.15,shuffle=True,seed=66)
elif dataset_arg=="_reasoning":
    en_dataset=load_from_disk("/home/ckqsudo/code2024/Mistral-7b-finetune/hf_en_reasoning_dataset_3")
    dataset=en_dataset.train_test_split(0.15,shuffle=True,seed=66)
elif dataset_arg in ["_pure_dialog","_continue_training"]:
    en_dataset=load_from_disk("/home/ckqsudo/code2024/Mistral-7b-finetune/hf_en_pure_dialog_dataset_3")
    dataset=en_dataset.train_test_split(0.15,shuffle=True,seed=66)
elif dataset_arg=="_pure_reasoning_observation":
    en_obs=load_from_disk("/home/ckqsudo/code2024/Mistral-7b-finetune/hf_en_pure_observation_dataset_3")
    en_rea=load_from_disk("/home/ckqsudo/code2024/Mistral-7b-finetune/hf_en_pure_reasoning_dataset_3")
    obs_dataset=en_obs.train_test_split(0.15,shuffle=True,seed=66)
    rea_dataset=en_rea.train_test_split(0.15,shuffle=True,seed=66)
    en_dataset = concatenate_datasets(dsets=[obs_dataset['train'],rea_dataset['train']]).shuffle(66)
    print(en_dataset)
    dataset=DatasetDict()
    dataset["train"],dataset["test"]=en_dataset,concatenate_datasets(dsets=[obs_dataset['test'],rea_dataset['test']])
else:
    raise ValueError()
dataset["train"].to_json(f"train_and_test_data/train_dataset{dataset_arg}.jsonl",orient="records")
dataset["test"].to_json(f"train_and_test_data/test_dataset{dataset_arg}.jsonl", orient="records")
train_dataset = load_dataset("json",data_files=f"train_and_test_data/train_dataset{dataset_arg}.jsonl")
test_dataset= load_dataset("json",data_files=f"train_and_test_data/test_dataset{dataset_arg}.jsonl")
print(dataset_arg+dataset_arg+dataset_arg+dataset_arg)

# test_dataset = load_dataset("json", data_files="test_dataset.json")

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

_continue_training_continue_training_continue_training_continue_training


In [6]:
dataset_arg

'_continue_training'

In [7]:
[train_dataset,test_dataset]

[DatasetDict({
     train: Dataset({
         features: ['messages'],
         num_rows: 456
     })
 }),
 DatasetDict({
     train: Dataset({
         features: ['messages'],
         num_rows: 81
     })
 })]

In [52]:
# def valid_messages_format(tokenizer,train_dataset,test_dataset):
#     check_list=[]
#     dataset_dict_list=[train_dataset,test_dataset]
#     for ds_dict in dataset_dict_list:
#         for item in ds_dict['train']['messages']:
#             check_str=""
#             for idx,dialog_item in enumerate(item):
#                 check_str+=dialog_item["role"]
#             tokenizer.apply_chat_template(item, tokenize=False)
#             if check_str.startswith('user')==False:
#                 print(check_str)
#             if 'useruser' in check_str:
#                 print(check_str)
# valid_messages_format(tokenizer,train_dataset,test_dataset)

In [7]:
# token_ids = tokenizer("[INST]", add_special_tokens=False)
# print(token_ids['input_ids'])


In [8]:
model_path="/home/ckqsudo/code2024/0model/Mistral-7B-Instruct-v0.2"

In [9]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from trl import setup_chat_format
from peft import LoraConfig, PeftModel
# tokenizer = AutoTokenizer.from_pretrained(model_path,padding="longest")
# Hugging Face model id
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)
# Load model and tokenizer

# Load model and tokenizer


if dataset_arg=='_continue_training':
    print("merge_model")
    base_model = AutoModelForCausalLM.from_pretrained(
        model_path,
        low_cpu_mem_usage=True,
        return_dict=True,
        torch_dtype=torch.bfloat16,
        device_map="auto")   
    merged_model= PeftModel.from_pretrained(base_model,model_id="/home/ckqsudo/code2024/Mistral-7b-finetune/trl_standard_output/fuck8_6_8_unpacking__pure_reasoning_observation/checkpoint-210")
    merged_model = merged_model.merge_and_unload()
    merged_model.save_pretrained("/home/ckqsudo/code2024/Mistral-7b-finetune/trl_standard_output/merge_model_6_11/merge_model")
    model = AutoModelForCausalLM.from_pretrained(
        "/home/ckqsudo/code2024/Mistral-7b-finetune/trl_standard_output/merge_model_6_8/merge_model",
        device_map="auto",
        attn_implementation="flash_attention_2",
        torch_dtype=torch.bfloat16,
        quantization_config=bnb_config
    )
else:
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        device_map="auto",
        attn_implementation="flash_attention_2",
        torch_dtype=torch.bfloat16,
        quantization_config=bnb_config
    )
tokenizer = AutoTokenizer.from_pretrained(model_path,padding="longest")
# tokenizer = AutoTokenizer.from_pretrained(model_path,truncation=True,padding_side = 'right',padding="longest",padding_side='right')

# tokenizer.padding_side = 'right' # to prevent warnings
# tokenizer.pad_token = tokenizer.unk_token # 消融实验

merge_model


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

: 

In [10]:
model_path,tokenizer

('/home/ckqsudo/code2024/0model/Mistral-7B-Instruct-v0.2',
 LlamaTokenizerFast(name_or_path='/home/ckqsudo/code2024/0model/Mistral-7B-Instruct-v0.2', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
 	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 })

In [11]:
tokenizer.padding_side,tokenizer.pad_token_id,tokenizer.pad_token,tokenizer.unk_token_id

('left', None, None, 0)

In [12]:
len(train_dataset['train']["messages"])

456

In [58]:
# tokenizer.pad_token_id=-1

In [13]:
from peft import LoraConfig

# LoRA config based on QLoRA paper & Sebastian Raschka experiment

peft_config = LoraConfig(
    lora_alpha=128,
    lora_dropout=0.05,
    r=256,
    target_modules=["q_proj","k_proj","v_proj","o_proj","up_proj","down_proj","gate_proj","lm_head"],
    # target_modules="all-linear",有些情况下可以这样设置参数，但是一般有些问题
    bias="none",
    task_type="causal_lm",
)

In [14]:
# dataset["train"][0]
tokenizer._get_padding_truncation_strategies()


(<PaddingStrategy.DO_NOT_PAD: 'do_not_pad'>,
 <TruncationStrategy.DO_NOT_TRUNCATE: 'do_not_truncate'>,
 None,
 {})

In [15]:
a=tokenizer.apply_chat_template(dataset['train'][0]['messages'], tokenize=False)
len(dataset['train']),len(dataset['test']),dataset,type(a)

(456,
 81,
 DatasetDict({
     train: Dataset({
         features: ['messages'],
         num_rows: 456
     })
     test: Dataset({
         features: ['messages'],
         num_rows: 81
     })
 }),
 str)

In [16]:
"""超棒的trl预处理函数"""
"""好用的代码来自 https://github.com/sids07/LLM_finetuning/blob/dafacc3ed04016eb54ad846793c405e97f7e1d5f/trainer-api/utils.py#L54 好用的代码"""
from datasets import DatasetDict, load_dataset
from transformers import DataCollatorForLanguageModeling
from trl import DataCollatorForCompletionOnlyLM
import numpy as np
from functools import partial

def preprocess_batch(batch, tokenizer, max_length):
    """Preprocess a batch of inputs for the language model."""
    batch["input_ids"] = tokenizer(batch["text"], max_length=max_length, truncation=True).input_ids
    return batch

def preprocess_dataset(dataset, tokenizer, max_length= 4096, seed=66):
    # partial 是 Python 中的一个内置函数，它用于部分应用一个函数，即固定函数的某些参数，返回一个新的函数。
    _preprocessing_function = partial(
        preprocess_batch, max_length=max_length, tokenizer=tokenizer
    )
    # 对数据集的每列做这种映射
    ckq_dataset = dataset.map(
    lambda examples: {
        'text': tokenizer.apply_chat_template(conversation=examples['messages'], tokenize=False, add_generation_prompt=False)
    }
    )
    dataset = ckq_dataset.map(
        _preprocessing_function,
        batched=True,
        remove_columns=['messages', "text"],
    )
    # trl 最坑的地方必须要丢弃所有列

    dataset = dataset.filter(lambda rec: len(rec["input_ids"]) < max_length)
    dataset = dataset.shuffle(seed=seed)
# https://github.com/sids07/LLM_finetuning/blob/dafacc3ed04016eb54ad846793c405e97f7e1d5f/trainer-api/utils.py#L54 好用的代码
    return dataset
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
trl_train_dataset=preprocess_dataset(dataset['train'],tokenizer)
trl_test_dataset=preprocess_dataset(dataset['test'],tokenizer)


In [17]:
"""input 入trl的 数据分离函数还有用于做 PLM"""
"""测试PLM和自动标注attention的效果"""
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM
response_template = '[/INST]'
instruction_template = '[INST]'
# 自动将位于此区间的文本做PLW，这一点非常重要！！！！
data_collator = DataCollatorForCompletionOnlyLM(instruction_template=instruction_template, response_template=response_template, tokenizer=tokenizer)
# data_collator = DataCollatorForCompletionOnlyLM(tokenizer=tokenizer, mlm=False, return_tensors="pt", pad_to_multiple_of=8
        # )
dataloader = torch.utils.data.DataLoader(dataset=trl_test_dataset, 
                                         collate_fn=data_collator, 
                                         batch_size=10
                                        )
for batch in dataloader:
    # 每个batch 都有三列 分别为
    print(batch)
    torch.set_printoptions(profile="full")#将 torch 充分打印的工具
    # print(x) # prints the whole tensor
    print(batch['labels'][0],batch['input_ids'][0],batch['attention_mask'][0])
    torch.set_printoptions(profile="default") # 将 torch 显示形式还原
    break

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': tensor([[32000, 32000, 32000,  ..., 28748, 16289, 28793],
        [32000, 32000, 32000,  ..., 28748, 16289, 28793],
        [32000, 32000, 32000,  ..., 14120, 28804,     2],
        ...,
        [32000, 32000, 32000,  ...,  3519, 28804,     2],
        [32000, 32000, 32000,  ..., 28748, 16289, 28793],
        [32000, 32000, 32000,  ...,  1753, 28804,     2]]), 'attention_mask': tensor([[0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        ...,
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1]]), 'labels': tensor([[ -100,  -100,  -100,  ...,  -100,  -100,  -100],
        [ -100,  -100,  -100,  ...,  -100,  -100,  -100],
        [ -100,  -100,  -100,  ..., 14120, 28804,     2],
        ...,
        [ -100,  -100,  -100,  ...,  3519, 28804,     2],
        [ -100,  -100,  -100,  ...,  -100,  -100,  -100],
        [ -100,  -100,  -100,  ...,  1753, 28804,     2]])}
tensor([ -100

In [18]:
dataset_arg

'_continue_training'

In [19]:
from transformers import TrainingArguments
from trl import SFTConfig

sftConfig = SFTConfig(
    packing=False,
    bf16=True,
    bf16_full_eval=True,
    dataset_batch_size=8,
    logging_strategy="steps",
    logging_dir=f"/home/ckqsudo/code2024/Mistral-7b-finetune/trl_standard_output/fuck8_6_8_unpacking_{dataset_arg}/log",
    output_dir=f"/home/ckqsudo/code2024/Mistral-7b-finetune/trl_standard_output/fuck8_6_8_unpacking_{dataset_arg}", # directory to save and repository id
    num_train_epochs=5,                     # number of training epochs
    per_device_train_batch_size=2,          # batch size per device during training
    gradient_accumulation_steps=2,          # number of steps before performing a backward/update pass
    gradient_checkpointing=True,            # use gradient checkpointing to save memory
    optim="adamw_torch_fused",              # use fused adamw optimizer
    save_strategy="steps",                  # save checkpoint every epoch
    save_steps=20,
    learning_rate=5e-5,                     # learning rate, based on QLoRA paper
    max_grad_norm=0.3,                      # max gradient norm based on QLoRA paper
    warmup_ratio=0.03,                      # warmup ratio based on QLoRA paper
    lr_scheduler_type="cosine",           # use constant learning rate scheduler
    push_to_hub=False,                       # push model to hub
    report_to="tensorboard",               # report metrics to tensorboard
    local_rank=2,
    remove_unused_columns=False,
    max_seq_length=4096,
    do_train=True, 
    do_eval=True,
    evaluation_strategy="steps",
    eval_steps=10,
    eval_packing=False,
    
    logging_steps=10,                       # log every 10 steps
    dataset_kwargs={
        "add_special_tokens": False,  # We template with special tokens
        "append_concat_token": False, # No need to add additional separator token 这两个是最后新加的
    },
    )
sftConfig.evaluation_strategy,sftConfig.output_dir# 这一步必须有

(<IntervalStrategy.STEPS: 'steps'>,
 '/home/ckqsudo/code2024/Mistral-7b-finetune/trl_standard_output/fuck8_6_8_unpacking__continue_training')

In [ ]:
# model.config.use_cache = False
# trainer.train(resume_from_checkpoint="/content/latest_checkpoint/")

In [20]:
import token
from typing import Literal
from trl import SFTTrainer
from trl.trainer import ConstantLengthDataset, DataCollatorForCompletionOnlyLM
max_seq_length = 4096 # max sequence length for model and packing of the dataset
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.padding_side  = 'left'
trainer = SFTTrainer(
    tokenizer=tokenizer,
    data_collator=data_collator,
    packing=False,
    model=model,
    args=sftConfig,
    # save_embedding_layers=True,
    dataset_text_field='input_ids',
    train_dataset=trl_train_dataset,
    eval_dataset=trl_test_dataset,
    peft_config=peft_config,
)
trainer.can_return_loss=True
# https://discuss.huggingface.co/t/no-log-for-validation-loss-during-training-with-trainer/40094/2


/home/ckqsudo/sys_tool/miniconda/tmp/yes/envs/trl/lib/python3.9/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)


/home/ckqsudo/sys_tool/miniconda/tmp/yes/envs/trl/lib/python3.9/site-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `dataset_batch_size` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/ckqsudo/sys_tool/miniconda/tmp/yes/envs/trl/lib/python3.9/site-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/ckqsudo/sys_tool/miniconda/tmp/yes/envs/trl/lib/python3.9/site-packages/trl/trainer/sft_trainer.py:382: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [21]:
str(trainer._get_output_dir)

'<bound method Trainer._get_output_dir of <trl.trainer.sft_trainer.SFTTrainer object at 0x75051f59e9a0>>'

In [22]:
# start training, the model will be automatically saved to the hub and the output directory
trainer.train()
trainer.save_model()
# assert trainer.state.log_history[(-1)]["train_loss"] is not None
# assert trainer.state.log_history[0]["eval_loss"] is not None
# save model 


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [134,0,0], thread: [64,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [134,0,0], thread: [65,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [134,0,0], thread: [66,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [134,0,0], thread: [67,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [134,0,0], thread: [68,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [134,0,0], thread: [69,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [134,

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
print(sftConfig,data_collator,tokenizer)

In [251]:
ckq_dataset['input_ids'][1][2],ckq_dataset['input_ids'][1][3]
# 打印向量

(733, 16289)

In [269]:
tokenizer.padding_side,tokenizer.pad_token_id,tokenizer.pad_token,tokenizer.unk_token_id

('left', 0, '<unk>', 0)

KeyError: 'images'

In [270]:
def conversations_formatting_function(tokenizer: AutoTokenizer, messages_field: Literal["messages", "conversations"]):
    r"""
    更加鲁棒的函数，能对列表进行遍历 查找messages 和conversations是否有messages信息
    """
    def format_dataset(examples):
        if isinstance(examples[messages_field][0], list):
            output_texts = []
            for i in range(len(examples[messages_field])):
                output_texts.append(tokenizer.apply_chat_template(examples[messages_field][i], tokenize=False))
            return output_texts
        else:
            return tokenizer.apply_chat_template(examples[messages_field], tokenize=False) 
    return format_dataset

In [206]:
trainer.state.log_history

[]

In [ ]:
### 以后就不用那么麻烦了，直接计算参数！！！

In [ ]:
def compute_metrics(preds, targets):
    assert len(preds) == len(targets), (
        "The length of pred_responses should be equal to the length of "
        "target_responses. But received {} and {}.".format(len(preds), len(targets))
    )
    rouge = Rouge()
    bleu4 = BLEU(n_size=4)
    scores = []
    for pred, target in zip(preds, targets):
        try:
            score = rouge.get_scores(" ".join(pred), " ".join(target))
            scores.append([score[0]["rouge-1"]["f"], score[0]["rouge-2"]["f"], score[0]["rouge-l"]["f"]])
        except ValueError:
            scores.append([0, 0, 0])
        bleu4.add_inst(pred, [target])
    rouge1 = np.mean([i[0] for i in scores])
    rouge2 = np.mean([i[1] for i in scores])
    rougel = np.mean([i[2] for i in scores])

    rouge1 = round(rouge1, 4)
    rouge2 = round(rouge2, 4)
    rougel = round(rougel, 4)
    bleu4 = round(bleu4.score(), 4)
    return dict(
        rouge1=rouge1,
        rouge2=rouge2,
        rougel=rougel,
        bleu4=bleu4,
    )

In [ ]:
"""一个老哥写的能在做PWL的时候做packing的函数"""
"""https://github.com/huggingface/trl/issues/632#top"""
"""https://arxiv.org/abs/2401.13586 PLM真的重要的吗论文"""
def get_assistant_start_end_indices(messages, conversation_text):
    start_indices = []
    current_index = 0
    for message in messages:
        message_text = message["content"]
        match_index = conversation_text[current_index:].find(message_text)
        start_indices.append(current_index + match_index)
        current_index += match_index + len(message_text)
    end_indices = [len(conversation_text) if i == len(start_indices) - 1 else start_indices[i+1] for i, x in enumerate(start_indices)]
    roles = [message["role"] for message in messages]
    return [(s, e) for s, e, r in zip(start_indices, end_indices, roles) if r == "assistant"]

def get_masked_labels(conversation_ids, assistant_ranges):
    labels = []
    for id_, (id_s, id_e) in list(zip(conversation_ids["input_ids"], conversation_ids["offset_mapping"])):
        if any(id_s >= s and id_e <= e for s, e in assistant_ranges):
            yield id_
        else:
            yield -100

def tokenize_messages(messages, tokenizer, mask_inputs=True):
    conversation_text = tokenizer.apply_chat_template(conversation=messages, add_generation_prompt=True, tokenize=False)
    conversation_ids = tokenizer(conversation_text, return_offsets_mapping=mask_inputs)
    if mask_inputs:
        assistant_ranges = get_assistant_start_end_indices(messages, conversation_text)
        labels = get_masked_labels(conversation_ids, assistant_ranges)
        conversation_ids["labels"] = list(labels)
        
        del conversation_ids["offset_mapping"]
    else:
        conversation_ids["labels"] = conversation_ids["input_ids"]
    return conversation_ids
# ckq_dataset = train_dataset['train'].map(
#     lambda examples: {
#         'input_ids': tokenize_messages(messages=examples['messages'], tokenizer=tokenizer, mask_inputs=True)
#     }
# )
ckq_dataset = train_dataset['train'].map(
    lambda examples: {
        'text': tokenizer.apply_chat_template(conversation=examples['messages'], add_generation_prompt=False, tokenize=False)
    }
)